In [0]:
import pyspark.sql.functions as f

In [0]:
src_table_name="ref_diag"
src_database="conformednpii"
tgt_db_typ="mysql"


In [0]:
%pip install -r requirements.txt

In [0]:
def get_current_notebook_path():
    notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    return notebook_path

def get_environment_from_notebook_path():
    #note_book_path = get_current_notebook_path()
    #main_logger.info(f"Notebook path :{note_book_path}")
    if  spark.conf.get("spark.databricks.clusterUsageTags.clusterOwnerOrgId")=='4022166418081681':
        return 'PT'
    elif spark.conf.get("spark.databricks.clusterUsageTags.clusterOwnerOrgId")=='add production id':
        return 'prod'

def get_user_mysql_cred(mysql_env,secret_mgr_scope='ds-edw-prod'):
    if mysql_env.lower() in ['pt','prod']:
        mysql_ip=None #dbutils.secrets.get(scope=secret_mgr_scope, key='HOST_NAME')
        mysql_port=None #dbutils.secrets.get(scope=secret_mgr_scope, key='HOST_PORT')
        musql_username=None #dbutils.secrets.get(scope=secret_mgr_scope, key='USERNAME')
        mysql_password=None #dbutils.secrets.get(scope=secret_mgr_scope, key='PASSWORD')
        return (mysql_ip,mysql_port,musql_username,mysql_password)
    else:
        raise Exception(f"Invalid environment for mysql user credentials {mysql_env}")

def get_redshift_user_cred(redshift_env,secret_mgr_scope):
    if redshift_env.lower() in ['prod']:
        redshift_ip_user = None #dbutils.secrets.get(scope=secret_mgr_scope, key="redshift_talend_username")
        redshift_password = None #dbutils.secrets.get(scope=secret_mgr_scope, key="redshift_talend_password")
        return redshift_ip_user,redshift_password
    else:
        raise Exception(f"Invalid environment for redshift user credentials {mysql_env}")


In [0]:

import entry_point
import importlib
importlib.reload(entry_point)
environment = get_environment_from_notebook_path()

mysql_ip,mysql_port,mysql_username,mysql_password = get_user_mysql_cred(mysql_env=environment,secret_mgr_scope='ds-edw-prod')
redshift_user,redshift_pass = get_redshift_user_cred(redshift_env='prod',secret_mgr_scope='ds-edw-prod')


In [0]:

df=spark.table("udf_internal.udf_to_rdbms_sync").where(f"src_tbl='{src_table_name}'").select("udf_to_rdbms_sync_key","src_db","src_tbl","tgt_db_typ","tgt_db","tgt_tbl","merge_keys")

assert df.count() > 0, "No records found in udf_internal.udf_to_rdbms_sync"

# check last successful log entry and get watermark value


for row in df.collect():
    src_db=row["src_db"]
    src_tbl=row["src_tbl"]
    tgt_db_typ=row["tgt_db_typ"]
    tgt_db=row["tgt_db"]
    tgt_tbl=row["tgt_tbl"]
    merge_keys=row["merge_keys"]
    udf_to_rdbms_sync_key=row["udf_to_rdbms_sync_key"]
    
    watermark_value = spark.table("udf_internal.udf_to_rdbms_sync_log").where(f"udf_to_rdbms_sync_key='{udf_to_rdbms_sync_key}'").where("current_status='success'").agg({"watermark_value": "max"}).collect()[0][0]


    filter_condition= "1=1" 
    if watermark_value:
        filter_condition=f"udf_updated_dt >= '{watermark_value}'"
    rows=spark.read.format("delta").load(f"/Volumes/ptudfnpii/npii/{src_db}/reference/{tgt_tbl}/").where(filter_condition).count()
    sql_conn= (mysql_ip,mysql_port,mysql_username,mysql_password,redshift_user,redshift_pass)
    print(len(sql_conn))
    master_obj= entry_point.start_dbks_dms(row,sql_conn)
  
                                    




    